# AWS Web App
#### Sam Berkson
#### CPSC 325
#### 2/21/23

#### Learning Objectives:
* Creating a basic AWS-hosted and deployed web app
* Familiarity with AWS Amplify, Lambda, DynamoDB
* Building a REST API on AWS

#### Pre-Requisites:
* AWS Free Tier Account
    * A credit card is needed to create an account, but we will not incur any charges with our demo.
    * https://aws.amazon.com/free/?trk=78b916d7-7c94-4cab-98d9-0ce5e648dd5f&sc_channel=ps&s_kwcid=AL!4422!3!432339156165!e!!g!!aws%20account&ef_id=CjwKCAiA0cyfBhBREiwAAtStHLNH5zLgdcyxPUKMqkzdFrTT8QCUc5uudgHdBBOCMCkHtkaMP6Sh5hoCyNkQAvD_BwE:G:s&s_kwcid=AL!4422!3!432339156165!e!!g!!aws%20account&all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsf.Free%20Tier%20Types=*all&awsf.Free%20Tier%20Categories=*all


* Working Web Browser

### Step 1: Create our Web App
* Platform: AWS Amplify
* Steps:
    * Open AWS Amplify
        * https://us-west-2.console.aws.amazon.com/amplify/home?region=us-west-2#/
    * Get Started > Host your web app
        * Deploy without Git Provider
        * Environment name: dev
        * Drag and drop index.html.zip
        * Save and deploy
    * Test Web App
        * Select Domain Management in left navigation menu
        * Copy and paste URL to see the rendered index.html code!

### Step 2: Build a Serverless Function
* Platform: AWS Lambda
* Steps:
    * Log into AWS Lambda console
        * https://console.aws.amazon.com/lambda/
    * Create Function
        * Name: HelloWorldFunction
        * Runtime: Python 3.9
    * Insert code into function and save. Deploy!

In [ ]:
# import the JSON utility package since we will be working with a JSON object
import json
# define the handler function that the Lambda service will use as an entry point
def lambda_handler(event, context):
    # extract values from the event object we got from the Lambda service
    name = event['firstName'] +' '+ event['lastName']
    # return a properly formatted JSON object
    return {
    'statusCode': 200,
    'body': json.dumps('Hello from Lambda, ' + name)
    }

* Test function
    * Click test
    * Give test event a name
    * Insert test code and save
    * Under test, select our function.  Status == 200 is a success!

In [ ]:
{
  "firstName": "Ada",
  "lastName": "Lovelace"
}

### Step 3: Link a Serverless Function to a Web App
* Platform: API Gateway
* Steps: 
    * Log into API Gateway console
        * https://us-west-2.console.aws.amazon.com/apigateway/main/apis?region=us-west-2
    * Select REST API and choose Build
        * Protocol: REST
    * Create new API
        * Name: HelloWorldAPI
        * Endpoint: Edge optimized
        * Create API
    * Create a new resource and method
        * API:HelloWorldAPI > Resources
        * Actions > Create Method
        * POST, confirm
        * Lambda Function integration type
        * Select our HelloWorld function
        * Actions > Enable CORS (Cross-origin resource sharing - allows us to use resources on a different domain)
    * Deploy API
        * Actions > Deploy
        * New Stage: dev
        * Deploy!
        * Take note of the Invoke URL:
    * Validate API
        * Resources > POST
        * Test
        * Input test code and test!

In [ ]:
{
    "firstName":"Grace",
    "lastName":"Hopper"
}

### Step 4: Create a Data Table
* Platform: DynamoDB
* Steps: 
    * Log into Amazon Dynamo DB console
        * https://console.aws.amazon.com/dynamodb/home
    * Create Table
        * Name: HelloWorldDatabase
        * Partition Key: ID
        * Create!
        * Take note of the Amazon Resource Name (ARN), this allows us to use this database!
        * ARN:
    * Create and add IAM policy to Lambda function
        * Open AWS Lambda Console
            * https://console.aws.amazon.com/lambda/
        * Select our function
        * Configuration > Permissions
        * Click Execution Role link
        * Add Permissions > Create inline policy
        * Paste JSON text
        * Replace "YOUR-TABLE-ARN" with the ARN of our DyanmoDB table
        * Review and name!
            * Name: HelloWorldDynamoPolicy

In [ ]:
{
"Version": "2012-10-17",
"Statement": [
    {
        "Sid": "VisualEditor0",
        "Effect": "Allow",
        "Action": [
            "dynamodb:PutItem",
            "dynamodb:DeleteItem",
            "dynamodb:GetItem",
            "dynamodb:Scan",
            "dynamodb:Query",
            "dynamodb:UpdateItem"
        ],
        "Resource": "YOUR-TABLE-ARN"
    }
    ]
}

* Modify our Lambda function to write to our DynamoDB table
    * Select Code tab in Lambda console
    * Paste updated code

In [ ]:
# import the json utility package since we will be working with a JSON object
import json
# import the AWS SDK (for Python the package name is boto3)
import boto3
# import two packages to help us with dates and date formatting
from time import gmtime, strftime

# create a DynamoDB object using the AWS SDK
dynamodb = boto3.resource('dynamodb')
# use the DynamoDB object to select our table
table = dynamodb.Table('HelloWorldDatabase')
# store the current time in a human readable format in a variable
now = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())

# define the handler function that the Lambda service will use as an entry point
def lambda_handler(event, context):
    # extract values from the event object we got from the Lambda service and store in a variable
    name = event['firstName'] +' '+ event['lastName']
    # write name and time to the DynamoDB table using the object we instantiated and save response in a variable
    response = table.put_item(
        Item={
            'ID': name,
            'LatestGreetingTime':now
            })
    # return a properly formatted JSON object
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda, ' + name)
    }

* Deploy!
* Test our changes
    * Open DynamoDB console
    * Tables > Explore items
    * Select our database and select items

### Step 5: Add Interactivity to Web App
* Platform: AWS Amplify
* Steps:
    * Open index-2.html 
        * On line 41, replace the url with API invoke URL
    * Compress and upload
    * Deploy and test!